In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 54.1 MB/s eta 0:00:00


In [6]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Datos del problema
values = [10, 12, 8, 5, 8, 5, 6, 7, 6, 12, 8, 8, 9, 8, 9, 8, 3, 7, 8, 5, 6]
weights = [6, 7, 7, 3, 5, 2, 4, 5, 9, 3, 9, 8, 7, 8, 6, 5, 2, 3, 5, 4, 6]
K = 50
n = len(values)

# Definir el problema
model = LpProblem(name="knapsack-problem", sense=LpMaximize)

# Definir las variables de decisión (0 o 1 para cada objeto)
x = [LpVariable(f"x{i}", cat="Binary") for i in range(n)]

# Función objetivo: maximizar el valor total
model += lpSum(values[i] * x[i] for i in range(n)), "Total_Value"

# Restricción de capacidad: el peso total no debe exceder K
model += lpSum(weights[i] * x[i] for i in range(n)) <= K, "Capacity"

# Resolver el problema
model.solve()

# Resultados de programación lineal
selected_items_lp = [i + 1 for i in range(n) if x[i].value() == 1]
total_value_lp = sum(values[i] for i in range(n) if x[i].value() == 1)
total_weight_lp = sum(weights[i] for i in range(n) if x[i].value() == 1)

print("Resultados de Programación Lineal:")
print("Selección óptima de objetos:", selected_items_lp)
print("Valor total de la recompensa:", total_value_lp)
print("Peso total de la selección:", total_weight_lp)

Resultados de Programación Lineal:
Selección óptima de objetos: [1, 2, 4, 5, 6, 7, 8, 10, 16, 17, 18, 19]
Valor total de la recompensa: 91
Peso total de la selección: 50


In [5]:
import random
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Datos del problema
values = [10, 12, 8, 5, 8, 5, 6, 7, 6, 12, 8, 8, 9, 8, 9, 8, 3, 7, 8, 5, 6]
weights = [6, 7, 7, 3, 5, 2, 4, 5, 9, 3, 9, 8, 7, 8, 6, 5, 2, 3, 5, 4, 6]
K = 50
n = len(values)

# Definir el problema
model = LpProblem(name="knapsack-problem", sense=LpMaximize)

# Definir las variables de decisión (0 o 1 para cada objeto)
x = [LpVariable(f"x{i}", cat="Binary") for i in range(n)]

# Función objetivo: maximizar el valor total
model += lpSum(values[i] * x[i] for i in range(n)), "Total_Value"

# Restricción de capacidad: el peso total no debe exceder K
model += lpSum(weights[i] * x[i] for i in range(n)) <= K, "Capacity"

# Resolver el problema
model.solve()

# Resultados de programación lineal
selected_items_lp = [i + 1 for i in range(n) if x[i].value() == 1]
total_value_lp = sum(values[i] for i in range(n) if x[i].value() == 1)
total_weight_lp = sum(weights[i] for i in range(n) if x[i].value() == 1)

print("Resultados de Programación Lineal:")
print("Selección óptima de objetos:", selected_items_lp)
print("Valor total de la recompensa:", total_value_lp)
print("Peso total de la selección:", total_weight_lp)

# Algoritmo Genético
# Parámetros del algoritmo genético
N = 50    # Tamaño de la población
s = 0.5   # % de selección (50%)
c = 0.8   # % de cruce (80%)
m = 0.1   # % de mutación (10%)
maxI = 200  # Máximo número de iteraciones

# Funciones auxiliares
def fitness(individual):
    total_value = sum(individual[i] * values[i] for i in range(len(values)))
    total_weight = sum(individual[i] * weights[i] for i in range(len(weights)))
    if total_weight > K:
        return 0  # Penalización si excede la capacidad
    return total_value

def selection(population):
    population = sorted(population, key=fitness, reverse=True)
    return population[:int(s * len(population))]

def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual):
    for i in range(len(individual)):
        if random.random() < m:
            individual[i] = 1 - individual[i]  # Cambia de 0 a 1 o de 1 a 0
    return individual

# Inicialización de la población
population = [[random.randint(0, 1) for _ in range(len(values))] for _ in range(N)]

# Algoritmo genético
for iteration in range(maxI):
    # Evaluación y selección
    population = selection(population)

    # Cruce
    new_population = []
    while len(new_population) < N * c:
        parent1, parent2 = random.sample(population, 2)
        child1, child2 = crossover(parent1, parent2)
        new_population.extend([child1, child2])

    # Mutación
    population += [mutate(individual) for individual in new_population]

    # Rellenar la población si es necesario
    while len(population) < N:
        population.append([random.randint(0, 1) for _ in range(len(values))])

# Obtener la mejor solución
best_solution = max(population, key=fitness)
best_value = fitness(best_solution)
best_weight = sum(best_solution[i] * weights[i] for i in range(len(weights)))

print("\nResultados del Algoritmo Genético:")
selected_items_ga = [i + 1 for i in range(n) if best_solution[i] == 1]
print("Selección óptima de objetos:", selected_items_ga)
print("Valor total de la recompensa:", best_value)
print("Peso total de la selección:", best_weight)

# Comparación de resultados
print("\nComparación de Resultados:")
print("¿Es la misma solución?", selected_items_lp == selected_items_ga)
print("¿Es mejor o peor la solución del algoritmo genético?", "Mejor" if best_value > total_value_lp else "Peor" if best_value < total_value_lp else "Igual")


Resultados de Programación Lineal:
Selección óptima de objetos: [1, 2, 4, 5, 6, 7, 8, 10, 16, 17, 18, 19]
Valor total de la recompensa: 91
Peso total de la selección: 50

Resultados del Algoritmo Genético:
Selección óptima de objetos: [1, 2, 4, 5, 6, 7, 8, 10, 16, 17, 18, 19]
Valor total de la recompensa: 91
Peso total de la selección: 50

Comparación de Resultados:
¿Es la misma solución? True
¿Es mejor o peor la solución del algoritmo genético? Igual
